In [48]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

import zipfile

In [3]:
full_price = pd.read_csv("/Users/cindy/quandl_data_table_downloads/QUOTEMEDIA/QUOTEMEDIA_PRICES_20230614.csv")
full_price.head()

,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
0,JTKWY,2022-03-11,6.1700,7.3200,5.790,6.72,9440097.0,0.0,1.0,6.1700,7.3200,5.790,6.72,9440097.0
1,JTKWY,2022-03-10,6.1600,6.1750,5.935,6.07,2261623.0,0.0,1.0,6.1600,6.1750,5.935,6.07,2261623.0
2,FG_1,2020-06-01,8.1000,8.3900,8.100,8.39,3086317.0,0.0,1.0,8.1000,8.3900,8.100,8.39,3086317.0
3,FLWS,2022-03-09,14.5700,14.9588,14.410,14.45,662492.0,0.0,1.0,14.5700,14.9588,14.410,14.45,662492.0
4,RENW_,2020-01-29,21.9768,21.9900,21.970,21.99,319.0,0.0,1.0,21.9768,21.9900,21.970,21.99,319.0


In [5]:
SP500 = pd.read_excel("S&P 500 tickers.xlsx")
SP500.head()

,Company,Symbol,Weight
0,Apple Inc.,AAPL,6.634317
1,Microsoft Corporation,MSFT,5.627977
2,Amazon.com Inc.,AMZN,2.508250
3,Alphabet Inc. Class A,GOOGL,1.638209
4,Berkshire Hathaway Inc. Class B,BRK.B,1.629718


In [59]:
F_F_5_Factors = pd.read_csv("F_F_Research_Data_5_Factors_daily.csv", skiprows = 3)
F_F_5_Factors.columns = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
F_F_5_Factors['Date'] = pd.to_datetime(F_F_5_Factors['Date'], format='%Y%m%d')
F_F_5_Factors.head()

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,1963-07-01,-0.67,0.02,-0.35,0.03,0.13,0.012
1,1963-07-02,0.79,-0.28,0.28,-0.08,-0.21,0.012
2,1963-07-03,0.63,-0.18,-0.10,0.13,-0.25,0.012
3,1963-07-05,0.40,0.09,-0.28,0.07,-0.30,0.012
4,1963-07-08,-0.63,0.07,-0.20,-0.27,0.06,0.012


We define the universe as newest S&P 500 stocks that have at least 5 years of price data. It means that we will exclude the stocks that are IPO after 2018-01-01.

In [10]:
# Only keep the stocks that are in the S&P 500
universe = full_price[full_price['ticker'].isin(SP500['Symbol'])].copy().sort_values(by = ['ticker', 'date'])
universe['date'] = pd.to_datetime(universe['date'])
universe

,ticker,date,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
34609545,A,1999-11-18,45.50,50.00,40.00,44.00,44739900.0,0.0,1.0,29.783619,32.729252,26.183402,28.801742,44739900.0
8276274,A,1999-11-19,42.94,43.00,39.81,40.38,10897100.0,0.0,1.0,28.107882,28.147157,26.059030,26.432144,10897100.0
8276273,A,1999-11-22,41.31,44.00,40.06,44.00,4705200.0,0.0,1.0,27.040908,28.801742,26.222677,28.801742,4705200.0
8276272,A,1999-11-23,42.50,43.63,40.25,40.25,4274400.0,0.0,1.0,27.819864,28.559545,26.347048,26.347048,4274400.0
8276271,A,1999-11-24,40.13,41.94,40.00,41.06,3464400.0,0.0,1.0,26.268498,27.453297,26.183402,26.877262,3464400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5882272,ZTS,2023-06-07,170.14,170.14,162.84,163.58,3195076.0,0.0,1.0,170.140000,170.140000,162.840000,163.580000,3195076.0
6750729,ZTS,2023-06-08,162.96,164.87,162.38,164.30,1983997.0,0.0,1.0,162.960000,164.870000,162.380000,164.300000,1983997.0
45051669,ZTS,2023-06-09,163.73,164.23,162.95,163.09,1346569.0,0.0,1.0,163.730000,164.230000,162.950000,163.090000,1346569.0
17458601,ZTS,2023-06-12,163.84,165.61,162.76,165.55,1422996.0,0.0,1.0,163.840000,165.610000,162.760000,165.550000,1422996.0


In [42]:
# Filter out the stocks that are IPO after 2018-01-01
SP500_start_date = universe.groupby('ticker').date.min().to_frame().reset_index().rename(columns = {'date': 'start_date'})
SP500_start_date[SP500_start_date.start_date >= '2018-01-01'].ticker

28     AMCR
75     CARR
82     CDAY
119    CTVA
137     DOW
227     HWM
270     LHX
311    MRNA
344     OGN
350    OTIS
447      TT
463    VICI
Name: ticker, dtype: object

In [43]:
universe.drop(universe[(universe.ticker.isin(SP500_start_date[SP500_start_date.start_date >= '2018-01-01'].ticker))].index, inplace = True)

In [44]:
# Check the start date of each stock is earlier than 2018-01-01
#SP500_start_date = universe.groupby('ticker').date.min().to_frame().reset_index().rename(columns = {'date': 'start_date'})
#SP500_start_date[SP500_start_date.start_date == SP500_start_date.start_date.max()]

,ticker,start_date
121,DD,2017-09-01


In [73]:
# Set the start time be 2018-01-01 and end time be 2023-05-31 of the time series and compute returns
universe = universe[(universe['date'] >= '2018-01-01') & (universe['date'] < '2023-06-01')].sort_values(by = ['ticker', 'date'])[['ticker', 'date', 'adj_close', 'adj_volume']].reset_index(drop = True)
universe['return'] = universe.groupby('ticker')['adj_close'].pct_change()
universe = universe.dropna()
universe.head()

,ticker,date,adj_close,adj_volume,return
1,A,2018-01-09,69.013902,2666711.0,0.024554
2,A,2018-01-10,68.071536,2957184.0,-0.013655
3,A,2018-01-11,68.081152,1511134.0,0.000141
4,A,2018-01-12,68.975438,1448155.0,0.013136
5,A,2018-01-16,68.494639,1703398.0,-0.006971


In [74]:
# Match the date of Fama-French 5 factors to the universe date time series
F_F_5_Factors = F_F_5_Factors[(F_F_5_Factors['Date'] >= '2018-01-01') & (F_F_5_Factors['Date'] < '2023-06-01')].reset_index(drop = True)
F_F_5_Factors

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,2018-01-02,0.85,0.31,-0.21,-0.50,0.31,0.006
1,2018-01-03,0.59,-0.47,-0.19,-0.70,-0.06,0.006
2,2018-01-04,0.42,-0.22,0.25,-0.01,0.31,0.006
3,2018-01-05,0.66,-0.37,-0.25,0.37,-0.37,0.006
4,2018-01-08,0.19,-0.19,0.04,-0.07,0.01,0.006
...,...,...,...,...,...,...,...
1335,2023-04-24,0.00,-0.27,0.47,0.66,0.28,0.018
1336,2023-04-25,-1.76,-0.83,0.10,0.44,0.89,0.018
1337,2023-04-26,-0.41,-0.04,-0.75,0.05,-0.33,0.018
1338,2023-04-27,1.85,-0.59,0.00,0.46,-0.19,0.018


In [78]:
F_F_regression = universe.merge(F_F_5_Factors, left_on = 'date', right_on = 'Date', how = 'left')
F_F_regression['excess_return'] = F_F_regression['return'] - F_F_regression['RF']
F_F_regression

,ticker,date,adj_close,adj_volume,return,Date,Mkt-RF,SMB,HML,RMW,CMA,RF,excess_return
0,A,2018-01-09,69.013902,2666711.0,0.024554,2018-01-09,0.15,-0.39,-0.04,-0.13,0.02,0.006,0.018554
1,A,2018-01-10,68.071536,2957184.0,-0.013655,2018-01-10,-0.07,0.13,0.56,-0.32,0.02,0.006,-0.019655
2,A,2018-01-11,68.081152,1511134.0,0.000141,2018-01-11,0.87,1.12,0.30,-0.21,0.58,0.006,-0.005859
3,A,2018-01-12,68.975438,1448155.0,0.013136,2018-01-12,0.66,-0.33,0.16,-0.10,0.18,0.006,0.007136
4,A,2018-01-16,68.494639,1703398.0,-0.006971,2018-01-16,-0.49,-0.99,0.01,0.44,-0.40,0.006,-0.012971
...,...,...,...,...,...,...,...,...,...,...,...,...,...
659536,ZTS,2023-05-24,169.140000,2437851.0,-0.020444,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
659537,ZTS,2023-05-25,165.240000,3623624.0,-0.023058,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
659538,ZTS,2023-05-26,164.740000,1909375.0,-0.003026,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
659539,ZTS,2023-05-30,160.940000,2184974.0,-0.023067,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Using expanding window to compute the beta by regressing excess return on Mkt-RF, SMB, HML, RMW, CMA
F_F_regression['Mkt-RF_beta'] = F_F_regression.groupby('ticker')['excess_return'].apply(lambda x: x.expanding().cov(F_F_regression['Mkt-RF']) / F_F_regression['Mkt-RF'].expanding().var())

In [ ]:
F_F_regression